# TV Script Generation

In this project, we'll generate our own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs!

![](http://media.globaltv.com/uploadedimages/pages/shows/the_simpsons/the_simpsons_smartforms/mainelement.png?n=6428)

We'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  

The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).

## Get the Data

The data is already provided for you, but we'll just be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
import numpy as np

view_sentence_range = (0, np.random.randint(1, 50))

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 36:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician 

## Preprocess the Data

As usual, the first thing to do to any dataset is preprocessing.  Let's implement the following preprocessing functions below:

- Lookup Table
- Tokenize Punctuation

### Lookup Table

**To create a word embedding, you first need to transform the words to ids**. So, in this function, let's create two dictionaries:

- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    vocab = set(text) # Make sure there are no duplicate toknes
    vocab_to_int = {word:id for id, word in enumerate(vocab)} # enumerate and assign unique numeric id 
    int_to_vocab = {id:word for word, id in vocab_to_int.items()} # flip the dictionary
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, **punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!"**.

To avoid this, let's implement the function `token_lookup()` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  In fact, let's create a dictionary for the following symbols where the symbols are the key and value is the token:

- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it. The `||token||` notation ensures that the starts, stops, and other syntax of spoken english can be preserved in the model. 

It does this by directly separating the symbols as it's own word, making it easier for the neural network to predict on the next word; if a token could easily be confused as a word, that would lead to some particularly manged generational grammar. 

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # Literally a dictionary lookup
    return {'.':'||period||', 
            ',':'||comma||', 
            '"':'||double_quotes||',
            ';':'||semicolon||',
            '!':'||exclamation_mark||',
            '?':'||question_mark||',
            '(':'||left_paren||',
            ')':'||right_paren||',
            '--':'||dash||',
            '\n':'||newline||'}
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Save the Data
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here, since the preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:

- **`get_inputs()`:** Create TF Placeholders to recieve data, hold targets, and set the learning rate. 
- **`get_init_cell()`:** Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) into a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- **`get_embed()`:** Use our dictionary functions to embed our words into a vector.
- **`build_rnn()`:** Turn our LSTM `MultiRNNCell` cell structure into a Recurrent Neural Network using `tf.nn.dynamic_rnn)()`. 
- **`build_nn()`**: Join up the LSTM `MultiRNNCell` cells, but using no activation function for it's logits.
- **`get_batches()`**: Turn our input and target vectors in paired batches we can use to train the network.

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name())) # FIXME: macOS cuda dependency error

TensorFlow Version: 1.0.0


/Users/vivek.menon/Documents/udacity/projects/tv-script-generation/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input = tf.placeholder( tf.int32, [None, None], name='input') # Empty int placeholder 
    targets = tf.placeholder(tf.int32, [None, None], name='targets') # Empty int placeholder
    learning_rate = tf.placeholder(tf.float32, name='learning_rate') # Empty float placeholder
    return (input, targets, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


TensorFlow uses a **tensor data structure** to represent all data -- only tensors are passed between operations in the computation graph; essentially an n-dimensional array or list.

![](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/Images/enter-the-matrix-10-638.jpg)

A tensor has a **static type, a rank, and a shape**.

* *Rank*: The number of dimensions of a tensor. [Tensor rank](http://mathworld.wolfram.com/TensorRank.html) is not the same as matrix rank; but it can sometimes be referred to as it's order, degree, or n-dimensions. For example, the following tensor (defined as a Python list) has a rank of 2:

`t = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]`

For TensorFlow, rank basically refers to it's ability to have two lists: `[[],[],..].[[],[],[]]...`. Therefore, a *rank two tensor is what we typically think of as a matrix, a rank one tensor is a vector*, so we can access any element in a rank two tensor with the syntax `t[i, j]`, and for a rank three tensor we would need `t[i, j, k]`.

Fantastic! You've perfectly used "None"'s to adequately create a dynamic sized placeholder variables.

get_inputs function is correctly implemented. Seems that you have good grasp of TF Placeholder, I appreciate your efforts.
There is a very good course offered by Stanford University, CS 20SI: Tensorflow for Deep Learning Research. This would help you in further understanding of Tensorflow and its application.

### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size) # Create as many LSTM cells as required (256)
    cell = tf.contrib.rnn.MultiRNNCell([cell]*1) # Line them up into a MultiRNN Cell
    
    initial_state = cell.zero_state(batch_size, tf.float32) # Set the initial state of the RNNs to 0
    initial_state = tf.identity(initial_state, name= "initial_state") # Give it a name! 
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


![](http://cdn.collider.com/wp-content/uploads/starship-troopers.jpg)

Here's are some **excellent** resources on [RNN's](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) from some very heavy hitters.

### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :http://localhost:8888/notebooks/dlnd_tv_script_generation.ipynb#param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    #embedding = tf.contrib.layers.embed_sequence(ids=tf.placeholder(tf.int32, [vocab_size]), vocab_size=vocab_size, embed_dim=embed_dim)
    embedding = tf.Variable(tf.random_uniform( (vocab_size, embed_dim))) # Create the right shaped embedding space (our rnn_size)
    print(embedding)
    
    embed = tf.nn.embedding_lookup(embedding, input_data) # Embed our placeholder input data into our embedding variable
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tensor("Variable/read:0", shape=(27, 256), dtype=float32)
Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32) # Use our cells and input placeholder
    final_state = tf.identity(final_state, name="final_state") # Name it again!
    return (outputs, final_state)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


    "It might be necessary to add an Embedding layer just before the RNN to embed the words into a smaller dimensional space. Since the LSTM operates linearly over its input X, then linearly embedding the 1-of-K words to some embedding dimension D first corresponds to the original case you tried to get working here, except the matrix is factored through a rank D-dimensional bottleneck. If that makes sense.

    Also, it is common to keep track of frequency of all words and discard words that appear, say, less than 5 times in the dataset." 
    
From [Andrej Karpathy](https://twitter.com/karpathy?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor). 

### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embedded_inputs = get_embed(input_data, vocab_size, rnn_size) # Embed our input data (of length vocab_size) into an rnn-shaped space
    outputs, final_state = build_rnn(cell, embedded_inputs) # Build our rnn using Multicell LSTM and our embedded inputs
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None) # Connect our layers, linking vocab_size with our outputs, and a linear activation fn
    
    return (logits, final_state)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tensor("Variable/read:0", shape=(27, 256), dtype=float32)
Tests Passed


### Batches

Now let's implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a `numpy` array with the shape: `(number of batches, 2, batch size, sequence length)`. 

This is because each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

Also, if we can't fill the last batch with enough data, we should drop the last batch (otherwise our shapes will be broken). 

For example:
`get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` should attempt to put this list into batches of 3 entries (each with a pair of 2 words), slicing off the entries that don't fit into this multiple of 3 system (`19, 20`). 

i.e. a `numpy` array like the following:

```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """

    n_batches = len(int_text) // (batch_size * seq_length) # How many batches do we need? Floor for int

    # Drop the last few characters to make only full batches
    x_vector = np.array(int_text[: n_batches * batch_size * seq_length]) # From start: last batch * batch size * seq length in batch
    y_vector = np.array(int_text[1: n_batches * batch_size * seq_length + 1]) # From second to last++
    y_vector[-1] = x_vector[0] # Jk, last is now the first element in our data for wraparound
    
    x_matrix = x_vector.reshape(batch_size, -1) # Shape is inferred from the length of th array and remaining dims?!
    y_matrix = y_vector.reshape(batch_size, -1) # What sorcery is this

    x_tensor = np.split(x_matrix, n_batches, 1) # Whoah, just split reshaped the matrix into a lists of lists of sequences
    y_tensor = np.split(y_matrix, n_batches, 1) # We tensor now
    
    batches = np.array(list(zip(x_tensor, y_tensor))) # Batch Tensors up 
    
    return batches

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


In [14]:
## Original submission. Ineffecient; use native operations like splits instead of for loops
# def get_batches(int_text, batch_size, seq_length):
#     """
#     Return batches of input and target
#     :param int_text: Text with the words replaced by their ids
#     :param batch_size: The size of batch
#     :param seq_length: The length of sequence
#     :return: Batches as a Numpy array
#     """

#     n_batches = len(int_text) // (batch_size * seq_length) # How many batches do we need? Floor div for int
#     data_len = n_batches * batch_size * seq_length # How many entries are we dealing with? (4480)
    
#     inputs = int_text[:data_len] # Input sequence starts at entry 0
#     targets = int_text[1:data_len] # Target sequence starts at entry 1
#     targets.append(int_text[0]) # Append first value of input to last value of target
    
#     x_reshape = np.reshape(inputs, (batch_size,n_batches, seq_length))  # Let's reshape this long sequence into n batches of our desired length
#     y_reshape = np.reshape(targets, (batch_size,n_batches, seq_length)) # Same for the target vector
    
#     result = [] # Empty array for inserting
#     for i in range(n_batches):
#         x = x_reshape[:, i] # The i entry in the input vector
#         y = y_reshape[:, i] # Gets matched with i target vector
#         batch_row = [x, y] # And bundlded together in a list of lists
#         result.append(batch_row) #And appended ot our empty list
        
#     return np.array(result) # Let's return this result
    

# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
# tests.test_get_batches(get_batches)

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [18]:
# Number of Epochs
num_epochs = 100 

# Batch Size
batch_size = 128 

# RNN Size
rnn_size = 256

# Embedding Dimension Size
embed_dim = 256

# Sequence Length
seq_length = 12 

# Learning Rate
learning_rate = 0.01

# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

Tensor("Variable/read:0", shape=(6779, 256), dtype=float32)


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/44   train_loss = 8.820
Epoch   0 Batch   10/44   train_loss = 7.019
Epoch   0 Batch   20/44   train_loss = 6.793
Epoch   0 Batch   30/44   train_loss = 6.812
Epoch   0 Batch   40/44   train_loss = 6.601
Epoch   1 Batch    6/44   train_loss = 6.145
Epoch   1 Batch   16/44   train_loss = 6.332
Epoch   1 Batch   26/44   train_loss = 6.349
Epoch   1 Batch   36/44   train_loss = 6.349
Epoch   2 Batch    2/44   train_loss = 6.228
Epoch   2 Batch   12/44   train_loss = 6.248
Epoch   2 Batch   22/44   train_loss = 6.261
Epoch   2 Batch   32/44   train_loss = 6.278
Epoch   2 Batch   42/44   train_loss = 6.448
Epoch   3 Batch    8/44   train_loss = 6.304
Epoch   3 Batch   18/44   train_loss = 6.263
Epoch   3 Batch   28/44   train_loss = 6.319
Epoch   3 Batch   38/44   train_loss = 6.219
Epoch   4 Batch    4/44   train_loss = 6.078
Epoch   4 Batch   14/44   train_loss = 5.985
Epoch   4 Batch   24/44   train_loss = 5.758
Epoch   4 Batch   34/44   train_loss = 5.628
Epoch   5 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [21]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    
    input_tensor = loaded_graph.get_tensor_by_name('input:0') # These tensors are weighted from training; let's just yank them!
    initial_state_tensor = loaded_graph.get_tensor_by_name('initial_state:0') # Not sure where this name:int syntax came from though..
    final_state_tensor = loaded_graph.get_tensor_by_name('final_state:0') # Must be TF specific
    probs_tensor = loaded_graph.get_tensor_by_name('probs:0') # Certainly doesn't seem very Pythonic
    
    return input_tensor, initial_state_tensor, final_state_tensor, probs_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [22]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    
    index = np.random.choice(len(int_to_vocab), 1, p=probabilities)[0] # Using our probabilities, find the next int in the sequence
    predicted_word = int_to_vocab[index] # Convert that int to a string using a dict!
    
    return predicted_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [23]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: heck no-- that's true. that's the man who is from the forget-me-shot.
moe_szyslak:(too voice) oh yeah, uh... what all you know who i made that?
bart_simpson: yeah? look, we gotta celebrate! throw a guy who fixes things?
homer_simpson: wait on!
homer_simpson: lisa's mad at me and marge is mad.(really does you,(flips page) from no, i have kids?
voice: excuse me, the plaster's flaking again!
crowd:(chanting) barney! i believe my wife was madonna.
krusty_the_clown: a chance of backgammon.
bart_simpson:(singing) hello...
dr. _zander: with you poor soul thing i see. health in the car.
moe_szyslak: better set the morning? how 'bout lenny.(stands up coaster) this coaster's fine.
moe_szyslak: just ask anyone in this bar.
moe_szyslak: power off, einstein.
moe_szyslak: yeah! ooh, you've been taking ventriloquism lessons.(nervous laugh)
lenny_leonard:(singing,


# The TV Script is Nonsensical

It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text! 

In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!

As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.